In [2]:
from selenium import webdriver
import pandas as pd
import time
from tqdm import tqdm

# 멜론 차트 크롤링

In [34]:
driver=webdriver.Chrome('/Users/gunwook/Desktop/chromedriver')

In [100]:
driver.get('https://www.melon.com/chart/search/index.htm')

In [234]:
chart_1 = driver.find_elements_by_css_selector('#lst50')

In [214]:
singer = []
sing = []
like = []
album = []

In [235]:
# 1위에서 50위까지
for i in range(0,50):
    a = chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank02 > a')[0].text
    b = chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(4) > div > div > div.ellipsis.rank01 > span > strong > a')[0].text
    c = int(chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(5) > div > button > span.cnt')[0].text.replace(',',''))
    d = chart_1[i].find_elements_by_css_selector('#lst50 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank03 > a')[0].text
    singer.append(a)
    sing.append(b)
    like.append(c)
    album.append(d)

In [236]:
# chart_2로 가기
driver.find_elements_by_css_selector('#frm > div.paginate.chart_page > span > a')[0].click()

In [237]:
chart_2 = driver.find_elements_by_css_selector('#lst100')

In [238]:
# 51위부터 100위까지
for i in range(0,50):
    a = chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank02 > a')[0].text
    b = chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(4) > div > div > div.ellipsis.rank01 > span > strong > a')[0].text
    c = int(chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(5) > div > button > span.cnt')[0].text.replace(',',''))
    d = chart_2[i].find_elements_by_css_selector('#lst100 > td:nth-child(4) > div > div > div:nth-child(3) > div.ellipsis.rank03 > a')[0].text
    singer.append(a)
    sing.append(b)
    like.append(c)
    album.append(d)

In [240]:
driver.get('https://www.melon.com/search/total/index.htm?q=EXO&section=&searchGnbYn=Y&kkoSpl=Y&kkoDpType=&linkOrText=T&ipath=srch_form')

# 가온차트 크롤링

In [167]:
driver.get('http://gaonchart.co.kr/main/section/chart/online.gaon')

In [151]:
list1 = []
for i in tqdm(range(47, 16, -1)):
    # 주간 설정(range는 원하는만큼 지정)
    driver.find_elements_by_css_selector('#chart_week_select')[0].click()
    time = driver.find_elements_by_css_selector('#chart_week_select > option:nth-child({0})'.format(i))[0].text
    driver.find_elements_by_css_selector('#chart_week_select > option:nth-child({0})'.format(i))[0].click()
    
    
    rank = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr> td.ranking')
    chart_top_100 = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.subject > p:nth-child(1)')
    gaon_100_score = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.count > p')
    production_top_100 = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.production > p.pro')
    dist_top_100 = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.production > p.dist')
    artist = driver.find_elements_by_css_selector('#wrap > div.chart > table > tbody > tr > td.subject > p.singer')
    for a,b,c,d,e,f in zip(chart_top_100,gaon_100_score,production_top_100,dist_top_100,artist,rank):
        
        a = re.split('[(-)]+' ,a.text)[0]
        e = e.text.split('|')[0]
        if e == '(여자)아이들':
            pass
        else:
            e = re.split('[(-)]+' ,e)[0]
        b = int(b.text.replace(',',''))
        list1.append([a, c.text, d.text, e, time, b, f.text])
        



  0%|          | 0/31 [00:00<?, ?it/s]

  3%|▎         | 1/31 [00:17<08:53, 17.78s/it]

  6%|▋         | 2/31 [00:38<09:01, 18.68s/it]

 10%|▉         | 3/31 [00:57<08:42, 18.67s/it]

 13%|█▎        | 4/31 [01:16<08:29, 18.88s/it]

 16%|█▌        | 5/31 [01:36<08:16, 19.11s/it]

 19%|█▉        | 6/31 [01:55<07:56, 19.07s/it]

 23%|██▎       | 7/31 [02:13<07:32, 18.86s/it]

 26%|██▌       | 8/31 [02:31<07:04, 18.46s/it]

 29%|██▉       | 9/31 [02:48<06:39, 18.17s/it]

 32%|███▏      | 10/31 [03:07<06:25, 18.34s/it]

 35%|███▌      | 11/31 [03:27<06:16, 18.82s/it]

 39%|███▊      | 12/31 [03:45<05:56, 18.78s/it]

 42%|████▏     | 13/31 [04:08<05:57, 19.85s/it]

 45%|████▌     | 14/31 [04:30<05:47, 20.45s/it]

 48%|████▊     | 15/31 [04:52<05:37, 21.10s/it]

 52%|█████▏    | 16/31 [05:15<05:24, 21.61s/it]

 55%|█████▍    | 17/31 [05:38<05:07, 21.99s/it]

 58%|█████▊    | 18/31 [05:56<04:32, 20.96s/it]

 61%|██████▏   | 19/31 [06:16<04:05, 20.44s/it]

 65%|██████▍   | 20/31 [06:35<03:39,

In [152]:
df = pd.DataFrame(list1, columns = ['name','production','distributor','artist','time','score','rank'])

In [153]:
df

,name,production,distributor,artist,time,score,rank
0,사랑에 연습이 있었다면,"엔에스씨컴퍼니, 디원미디어",카카오 M,임재현,2019.06.02~2019.06.08,44009459,1
1,누구 없소,YG Entertainment,YG Plus,이하이,2019.06.02~2019.06.08,43074538,2
2,2002,Atlantic Records UK,Warner Music,Anne-Marie,2019.06.02~2019.06.08,41877109,3
3,너에게 못했던 내 마지막 말은,Stone Music Entertainment,지니뮤직,다비치,2019.06.02~2019.06.08,38165624,4
4,bad guy,"Darkroom, Interscope Records",Universal Music,Billie Eilish,2019.06.02~2019.06.08,33544212,5
...,...,...,...,...,...,...,...
6195,Rest,블루바이닐,Dreamus,백예린,2019.12.29~2020.01.04,2367026,196
6196,비가 내리는 날에는,C9엔터테인먼트,카카오 M,윤하,2019.12.29~2020.01.04,2358512,197
6197,Havana,Syco Music/Epic,Sony Music,Camila Cabello,2019.12.29~2020.01.04,2354139,198
6198,Dance The Night Away,JYP Entertainment,Dreamus,TWICE,2019.12.29~2020.01.04,2351662,199


# 지니차트 크롤링

In [168]:
driver.get('https://www.genie.co.kr')

In [169]:
genre = []
runtime = []
sex = []
active_type = []

In [12]:
list1 = set(list(df['artist'] + ',' + df['name']))

In [182]:
df['name_artist'] = df['artist'] + ',' + df['name']

In [173]:
# 노래 정보 뽑아오기
# 장르, 런타임, 성별, 활동유형
for i in tqdm(set(list1)):
    try:
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys(i)
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_element_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.link > a').click()
        gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(3) > span.value').text
        time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        if ':' in time:
            pass
        else:
            time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(5) > span.value').text
            gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        runtime.append(time)
        genre.append(gen)
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys(i)
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_elements_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.info > a.artist.ellipsis')[0].click()
        activity_type = driver.find_elements_by_css_selector('#body-content > div.artist-main-infos > div.info-zone > ul > li:nth-child(1) > span.value')[0].text.split('/')
        sex.append(activity_type[0])
        active_type.append(activity_type[1])
    except:
        i = i.split(',')
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys('{0}'.format(i[1]))
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_element_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.link > a').click()
        gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(3) > span.value').text
        time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        if ':' in time:
            pass
        else:
            time = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(5) > span.value').text
            gen = driver.find_element_by_css_selector('#body-content > div.song-main-infos > div.info-zone > ul > li:nth-child(4) > span.value').text
        genre.append(gen)
        runtime.append(time)
        elem = driver.find_element_by_id('sc-fd')
        elem.clear()
        elem.send_keys('{0}'.format(i[1]))
        driver.find_elements_by_css_selector('#frmGNB > fieldset > input.btn-submit')[0].click()
        driver.find_elements_by_css_selector('#body-content > div.search_song > div.search_result_detail > div > table > tbody > tr:nth-child(1) > td.info > a.artist.ellipsis')[0].click()
        activity_type = driver.find_elements_by_css_selector('#body-content > div.artist-main-infos > div.info-zone > ul > li:nth-child(1) > span.value')[0].text.split('/')
        sex.append(activity_type[0])
        active_type.append(activity_type[1])






  0%|          | 0/689 [00:00<?, ?it/s]



  0%|          | 1/689 [00:02<32:33,  2.84s/it]



  0%|          | 2/689 [00:05<32:10,  2.81s/it]



  0%|          | 3/689 [00:08<31:06,  2.72s/it]



  1%|          | 4/689 [00:10<29:43,  2.60s/it]



  1%|          | 5/689 [00:12<29:22,  2.58s/it]



  1%|          | 6/689 [00:15<27:46,  2.44s/it]



  1%|          | 7/689 [00:17<27:46,  2.44s/it]



  1%|          | 8/689 [00:19<27:34,  2.43s/it]



  1%|▏         | 9/689 [00:22<26:47,  2.36s/it]



  1%|▏         | 10/689 [00:24<26:32,  2.35s/it]



  2%|▏         | 11/689 [00:26<26:24,  2.34s/it]



  2%|▏         | 12/689 [00:29<26:21,  2.34s/it]



  2%|▏         | 13/689 [00:31<26:28,  2.35s/it]



  2%|▏         | 14/689 [00:33<25:18,  2.25s/it]



  2%|▏         | 15/689 [00:37<29:39,  2.64s/it]



  2%|▏         | 16/689 [00:39<28:36,  2.55s/it]



  2%|▏         | 17/689 [00:41<27:29,  2.45s/it]



  3%|▎         | 18/689 [00:43<26:40,  2.39s/it]



  3%|▎         | 19/689 [0

 37%|███▋      | 253/689 [09:47<18:23,  2.53s/it]



 37%|███▋      | 254/689 [09:50<18:05,  2.50s/it]



 37%|███▋      | 255/689 [09:52<18:29,  2.56s/it]



 37%|███▋      | 256/689 [09:55<18:21,  2.54s/it]



 37%|███▋      | 257/689 [09:57<18:12,  2.53s/it]



 37%|███▋      | 258/689 [09:59<17:17,  2.41s/it]



 38%|███▊      | 259/689 [10:02<17:27,  2.44s/it]



 38%|███▊      | 260/689 [10:04<16:23,  2.29s/it]



 38%|███▊      | 261/689 [10:06<15:48,  2.22s/it]



 38%|███▊      | 262/689 [10:08<16:17,  2.29s/it]



 38%|███▊      | 263/689 [10:11<16:50,  2.37s/it]



 38%|███▊      | 264/689 [10:13<16:40,  2.36s/it]



 38%|███▊      | 265/689 [10:15<16:15,  2.30s/it]



 39%|███▊      | 266/689 [10:17<15:38,  2.22s/it]



 39%|███▉      | 267/689 [10:20<16:00,  2.28s/it]



 39%|███▉      | 268/689 [10:22<16:04,  2.29s/it]



 39%|███▉      | 269/689 [10:24<16:12,  2.32s/it]



 39%|███▉      | 270/689 [10:27<15:49,  2.27s/it]



 39%|███▉      | 271/689 [10:30<17:15,  2.48s/

 73%|███████▎  | 505/689 [19:31<07:12,  2.35s/it]



 73%|███████▎  | 506/689 [19:34<07:10,  2.35s/it]



 74%|███████▎  | 507/689 [19:36<06:58,  2.30s/it]



 74%|███████▎  | 508/689 [19:38<06:47,  2.25s/it]



 74%|███████▍  | 509/689 [19:40<06:39,  2.22s/it]



 74%|███████▍  | 510/689 [19:42<06:43,  2.26s/it]



 74%|███████▍  | 511/689 [19:45<06:44,  2.27s/it]



 74%|███████▍  | 512/689 [19:47<06:41,  2.27s/it]



 74%|███████▍  | 513/689 [19:49<06:28,  2.21s/it]



 75%|███████▍  | 514/689 [19:51<06:39,  2.28s/it]



 75%|███████▍  | 515/689 [19:54<06:45,  2.33s/it]



 75%|███████▍  | 516/689 [19:56<06:48,  2.36s/it]



 75%|███████▌  | 517/689 [19:59<07:02,  2.46s/it]



 75%|███████▌  | 518/689 [20:01<06:54,  2.42s/it]



 75%|███████▌  | 519/689 [20:04<06:41,  2.36s/it]



 75%|███████▌  | 520/689 [20:06<06:35,  2.34s/it]



 76%|███████▌  | 521/689 [20:08<06:12,  2.22s/it]



 76%|███████▌  | 522/689 [20:10<06:13,  2.23s/it]



 76%|███████▌  | 523/689 [20:13<06:32,  2.37s/

In [ ]:
data = pd.DataFrame(list2,columns = ['name_artist'])

In [ ]:
data['genre'] = genre
data['runtime'] = runtime
data['sex'] = sex
data['active_type'] = active_type

In [ ]:
df = pd.merge(df,data,how='left',on='name_artist')
del df['name_artist']

In [188]:
df.to_csv('2019.06~2019.12.csv')

# 멜론 좋아요 뽑기

In [35]:
driver.get('https://www.melon.com/search/total/index.htm?q=%EC%82%AC%EB%9E%91%EC%97%90+%EC%97%B0%EC%8A%B5%EC%9D%B4+%EC%9E%88%EC%97%88%EB%8B%A4%EB%A9%B4&section=&linkOrText=T&ipath=srch_form')

In [36]:
likes = []

In [31]:
# 중간에 계속 팅기는 문제 발생. 
# 시간 텀 줘도 계속 팅기는 문제 생김
count = 0
for i in tqdm(list1):
    try:
        elem = driver.find_element_by_id('top_search')
        elem.clear()
        elem.send_keys(i)
        driver.find_element_by_css_selector('#header_wrap > div.wrap_search_field > fieldset > button.btn_icon.search_m > span').click()
        like = int(driver.find_element_by_css_selector('#frm_songList > div > table > tbody > tr:nth-child(1) > td:nth-child(6) > div > button > span.cnt').text.replace(',',''))
        likes.append(like)
        if count == 5:
            time.sleep(10)
            count = 0
        else:
            count += 1
    except:
        i = i.split(',')
        elem = driver.find_element_by_id('top_search')
        elem.clear()
        elem.send_keys('{0}'.format(i[1]))
        driver.find_element_by_css_selector('#header_wrap > div.wrap_search_field > fieldset > button.btn_icon.search_m > span').click()
        like = int(driver.find_element_by_css_selector('#frm_searchSong > div > table > tbody > tr:nth-child(1) > td:nth-child(6) > div > button > span.cnt').text.replace(',',''))
        likes.append(like)
        if count == 5:
            time.sleep(10)
            count = 0
        else:
            count += 1




  0%|          | 0/689 [00:00<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="top_search"]"}
  (Session info: chrome=81.0.4044.129)
